In [2]:
import os
import requests
from requests.auth import HTTPBasicAuth
import pdb
import pandas as pd
import io

os.environ['PL_API_KEY'] = "c7b9c030faa746c7bfa5b32bbd0ea0ff" #Planet API key


In [3]:
def doFilter():
    
    #pdb.set_trace()
    # the geo json geometry object we got from geojson.io
    geo_json_geometry = {
      "type": "Polygon",
      "coordinates": [
        [
          [
            133.58173370361325,
            -23.889604667707317
          ],
          [
            133.59649658203125,
            -23.889604667707317
          ],
          [
            133.59649658203125,
            -23.877361621215687
          ],
          [
            133.58173370361325,
            -23.877361621215687
          ],
          [
            133.58173370361325,
            -23.889604667707317
          ]
        ]
      ]
    }

    # filter for items the overlap with our chosen geometry
    geometry_filter = {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": geo_json_geometry
    }

    # filter images acquired in a certain date range
    date_range_filter = {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": "2001-07-01T00:00:00.000Z",
        "lte": "2018-08-01T00:00:00.000Z"
      }
    }

    # filter any images which are more than 50% clouds
    cloud_cover_filter = {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lte": 0.5
      }
    }

    # create a filter that combines our geo and date filters
    # could also use an "OrFilter"
    filteredData = {
      "type": "AndFilter",
      "config": [geometry_filter, date_range_filter, cloud_cover_filter]
    }
    
    #pdb.set_trace()

    return filteredData


In [4]:
def getData():
    
    data = doFilter()
    
    # Search API request object
    search_endpoint_request = {"item_types": ["REOrthoTile"],"filter": data}
    
    result = requests.post('https://api.planet.com/data/v1/quick-search',auth=HTTPBasicAuth(os.environ['PL_API_KEY'], ''),json=search_endpoint_request)
    
    
    
    return result
    
    
    
    
    
    
    
    

In [5]:
def doAnalysis():
    
    result = getData()
    
    j = result.json()
    
    features = j.get('features')
    
    itemId = []
    
    itemType = []
    
    assetType = []
    
    for i in features:
        
        itemId = i.get('id')
        
        properties = i.get('properties')
        
        itemType = properties.get('item_type')
        
        perm = i.get('_permissions')
        
        analytic = perm[0]
        
        assetType = 'analytic'
        
        response = doActivation(itemId, itemType,assetType)
        
        status = response.status_code
        
        print status
        
        pdb.set_trace()
        
        item_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(itemType, itemId)

        result = requests.get(item_url, auth=HTTPBasicAuth(os.environ['PL_API_KEY'], ''))
        
        download_url = result.json()[assetType]['location']
        
        vsicurl_url = '/vsicurl/' + download_url

        output_file = item_id + '_subarea.tif'
        
        gdal.Warp(output_file, vsicurl_url, dstSRS = 'EPSG:4326', cutlineDSName = 'smallArea.geojson', cropToCutline = True)
        
        

        
        
        
        print itemId
        
        print itemType
        
        print analytic
        
        
    
    
    

In [6]:
def doActivation(itemId, itemType,assetType):
    
    session = requests.Session()
    session.auth = (os.environ['PL_API_KEY'], '')
    
    # request an item
    item = session.get(("https://api.planet.com/data/v1/item-types/" +"{}/items/{}/assets/").format(itemType, itemId))

    # extract the activation url from the item for the desired asset
    item_activation_url = item.json()[assetType]["_links"]["activate"]

    # request activation
    response = session.post(item_activation_url)
    
    return response
    
    

In [7]:
doAnalysis()

429
> <ipython-input-5-3ac1ac0673be>(37)doAnalysis()
-> item_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(itemType, itemId)
(Pdb) n
> <ipython-input-5-3ac1ac0673be>(39)doAnalysis()
-> result = requests.get(item_url, auth=HTTPBasicAuth(os.environ['PL_API_KEY'], ''))
(Pdb) n
> <ipython-input-5-3ac1ac0673be>(41)doAnalysis()
-> download_url = result.json()[assetType]['location']
(Pdb) result
<Response [200]>
(Pdb) q


BdbQuit: 

In [ ]:
j = result.json()

In [ ]:
j.keys()

In [ ]:
aa = j.get('features')

In [ ]:
bb = aa[0]

In [ ]:
bb.keys()

In [ ]:
bb.get('_links')

In [ ]:
bb

In [ ]:
type(aa)

In [ ]:
aa[2]